In [1]:
import tkinter as tk
import cv2
import mediapipe as mp
from PIL import Image, ImageTk
import subprocess
import time
class CameraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Hand Detection")
        self.root.geometry("1280x720")  # Set window size

        self.main_frame = tk.Frame(root)
        self.main_frame.pack(expand=True, fill=tk.BOTH)

        self.start_button = tk.Button(self.main_frame, text="Start", command=self.open_camera)
        self.start_button.pack(pady=10)

        self.stop_button = tk.Button(self.main_frame, text="Stop", command=self.stop_camera)
        self.stop_button.pack(pady=10)

        self.cap = None  # Initialize camera capture object
        self.frame_width = None
        self.frame_height = None

        self.hand_detector = mp.solutions.hands.Hands()
        self.drawing_utils = mp.solutions.drawing_utils

        self.status_label = tk.Label(self.main_frame, text="Hand Status: ")
        self.status_label.pack(pady=5)
        self.hand_status = tk.StringVar()
        self.hand_status.set("Unknown")
        self.status_display = tk.Label(self.main_frame, textvariable=self.hand_status)
        self.status_display.pack(pady=5)

        self.camera_label = tk.Label(self.main_frame)
        self.camera_label.pack()
        
        # Initialize debounce variables
        self.last_command_time = 0
        self.debounce_time = 1.0  # Adjust as needed (in seconds)
        
        
    def open_camera(self):
        if self.cap is None or not self.cap.isOpened():
            self.cap = cv2.VideoCapture(0)
            self.show_frame()
        else:
            print("Camera is already open.")

    def show_frame(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.flip(frame, 1)  # Flip frame horizontally for mirror effect
            frame = cv2.resize(frame, (960, 540))  # Resize frame
            self.frame_height, self.frame_width, _ = frame.shape
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect hands
            output = self.hand_detector.process(rgb_frame)
            hand_status = self.detect_hand_status(output)
            self.hand_status.set(hand_status)

            if hand_status == "Swiping Left":
                # Check if debounce time has passed since last command execution
                current_time = time.time()
                if current_time - self.last_command_time > self.debounce_time:
                    self.last_command_time = current_time
                    self.play_next()
                    time.sleep(1)  # Add a sleep timer after executing the command
                    #self.increase_volume()
                    
            elif hand_status == "Swiping Right":
                # Check if debounce time has passed since last command execution
                current_time = time.time()
                if current_time - self.last_command_time > self.debounce_time:
                    self.last_command_time = current_time
                    self.play_previous()
                    time.sleep(1)  # Add a sleep timer after executing the command
                    #self.increase_volume()
#             elif hand_status == "Peace":
#                 # Check if debounce time has passed since last command execution
#                 current_time = time.time()
#                 if current_time - self.last_command_time > self.debounce_time:
#                     self.last_command_time = current_time
#                     self.play_pause_spotify()
#                     time.sleep(1)  # Add a sleep timer after executing the command
#                     #self.increase_volume()   
            elif hand_status =="Closed":
                self.increase_volume()
            elif hand_status =="Open":
                self.decrease_volume()
                
            if output.multi_hand_landmarks:
                for hand_landmarks in output.multi_hand_landmarks:
                    self.drawing_utils.draw_landmarks(frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)

            # Convert the frame to ImageTk format and display it
            img = Image.fromarray(frame)
            img = ImageTk.PhotoImage(img)
            self.camera_label.configure(image=img)
            self.camera_label.image = img

            # Repeat the process
            self.camera_label.after(10, self.show_frame)
        else:
            print("Failed to capture frame.")

#     def detect_hand_status(self, output):
#         if output.multi_hand_landmarks:
#             for hand_landmarks in output.multi_hand_landmarks:
#                 landmarks = hand_landmarks.landmark
#                 thumb_tip = landmarks[4]
#                 index_tip = landmarks[8]
#                 thumb_y = thumb_tip.y * self.frame_height
#                 index_y = index_tip.y * self.frame_height

#                 # Calculate distance between thumb and index finger
#                 distance = abs(thumb_y - index_y)

#                 if distance < 50:
#                     return "Closed"
#                 else:
#                     return "Open"
#         return "Unknown"
#         def detect_hand_status(self, output):
#         if output.multi_hand_landmarks:
#             for hand_landmarks in output.multi_hand_landmarks:
#                 landmarks = hand_landmarks.landmark
#                 thumb_tip = landmarks[4]
#                 index_tip = landmarks[8]
#                 middle_tip = landmarks[12]  # Middle finger tip
#                 thumb_y = thumb_tip.y * self.frame_height
#                 index_y = index_tip.y * self.frame_height
#                 middle_y = middle_tip.y * self.frame_height

#                 # Calculate distance between thumb and index finger
#                 distance = abs(thumb_y - index_y)

#                 # Determine hand status based on thumb and index finger distance
#                 if distance < 50:
#                     return "Closed"
#                 elif middle_y < index_y and middle_y < thumb_y:  # Check if middle finger is above index and thumb
#                     return "Peace"  # Gesture for play/pause
#                 elif thumb_tip.x < index_tip.x:  # Check if thumb is to the left of the index finger
#                     return "Swiping Left"  # Gesture for play previous
#                 elif thumb_tip.x > index_tip.x:  # Check if thumb is to the right of the index finger
#                     return "Swiping Right"  # Gesture for play next

#         return "Unknown"
    
    
    def detect_hand_status(self, output):
        if output.multi_hand_landmarks:
            for hand_landmarks in output.multi_hand_landmarks:
                landmarks = hand_landmarks.landmark
                thumb_tip = landmarks[4]
                index_tip = landmarks[8]
                middle_tip = landmarks[12] 
                thumb_y = thumb_tip.y * self.frame_height
                index_y = index_tip.y * self.frame_height
                middle_y = middle_tip.y * self.frame_height

                # Calculate distance between thumb and index finger
                distance = abs(thumb_y - index_y)

                # Determine hand status based on thumb and index finger distance
                if distance < 50:
                    return "Closed"
                elif middle_y < index_y and middle_y < thumb_y:  # Check if middle finger is above index and thumb
                    return "Peace Sign"
                elif thumb_tip.x < index_tip.x:  # Check if thumb is to the left of the index finger
                    return "Swiping Left"  # Gesture for play previous
                elif thumb_tip.x > index_tip.x:  # Check if thumb is to the right of the index finger
                    return "Swiping Right"  # Gesture for play next
                else:
                    return "Open"  # Gesture for hand open

        return "Unknown"
    
    def increase_volume(self):
        subprocess.run(["osascript", "-e", "set volume output volume (output volume of (get volume settings) + 5)"])
    def decrease_volume(self):
        subprocess.run(["osascript", "-e", "set volume output volume (output volume of (get volume settings) - 5)"])
    # def play_previous():
    #     subprocess.run(["osascript", "-e", "tell application \"System Events\" to key code 97 using {fn down}"])
    def play_next(self):
        subprocess.run(["osascript", "-e", "tell application \"Spotify\" to next track"])

    def play_previous(self):
        subprocess.run(["osascript", "-e", "tell application \"Spotify\" to previous track"])
        
    def play_pause_spotify():
        subprocess.run(["osascript", "-e", "tell application \"Spotify\" to playpause"])

    
    def stop_camera(self):
        if self.cap is not None and self.cap.isOpened():
            self.cap.release()
        else:
            print("Camera is not open.")

root = tk.Tk()
app = CameraApp(root)
root.mainloop()

I0000 00:00:1709935664.999835 7704813 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Failed to capture frame.


In [3]:
pip install pynput 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyobjc

  Obtaining dependency information for pyobjc-framework-CoreServices==10.1 from https://files.pythonhosted.org/packages/b9/a2/40e3ea57d2db98dfe7673cf644da5f15d186f1d4d90ffc57ec19c1f71b20/pyobjc_framework_CoreServices-10.1-cp36-abi3-macosx_11_0_universal2.whl.metadata
  Using cached pyobjc_framework_CoreServices-10.1-cp36-abi3-macosx_11_0_universal2.whl.metadata (2.4 kB)


Using cached pyobjc_framework_CoreServices-10.1-cp36-abi3-macosx_11_0_universal2.whl (39 kB)
  Attempting uninstall: pyobjc-framework-CoreServices
    Found existing installation: pyobjc-framework-CoreServices 9.0
    Uninstalling pyobjc-framework-CoreServices-9.0:
      Successfully uninstalled pyobjc-framework-CoreServices-9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires pyqt5<5.16, which is not installed.
spyder 5.4.3 requires pyqtwebengine<5.16, which is not installed.
Note: you may need to restart the kernel to use updated packages.
